# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
df = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
products = pd.pivot_table(df, index = ['ProductName'], columns = ['CustomerID'], values = 'Quantity').fillna(0)
products.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
from scipy.spatial.distance import pdist, squareform
distances = squareform(pdist(products.T, 'euclidean'))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
distances_normalised = pd.DataFrame(1/(1 + distances), 
                         index=products.columns, columns=products.columns)

def similar_customers(customerID):
    return distances_normalised[customerID].sort_values(ascending=False)[1:6]

similar = similar_customers(33)
similar

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
def get_similar_products(customerID):
    similar = similar_customers(customerID)
    return pd.concat([df.loc[ID] for ID in similar.index.tolist()])

get_similar_products(33)

,Quantity
ProductName,
Apricots - Halves,1
Apricots Fresh,1
Bacardi Breezer - Tropical,1
Bagel - Plain,1
Banana - Leaves,1
...,...
Wine - Ej Gallo Sierra Valley,2
Wine - Pinot Noir Latour,1
Wine - Redchard Merritt,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
def potential_products(customerID):
    similars = get_similar_products(customerID)
    return similars.groupby('ProductName').agg({'Quantity':'sum'}).sort_values(by = 'Quantity',ascending = False)
potential_products(33)

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [13]:
def recommended_products(customerID):
    ranked = potential_products(customerID)
    merge = products[[33]].merge(ranked, right_on = 'ProductName', left_on = 'ProductName').sort_values(by = "Quantity", ascending = False)
    return merge.loc[merge[33] == 0.0].head()
recommended_products(33)

,33,Quantity
ProductName,,
Butter - Unsalted,0.0,3
Wine - Ej Gallo Sierra Valley,0.0,3
Soup - Campbells Bean Medley,0.0,3
Wine - Blue Nun Qualitatswein,0.0,3
Chicken - Soup Base,0.0,2


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [15]:
dictionary = dict()
customerIDS = set(data['CustomerID'].tolist())

def create_dictionary(customerID):
    dictionary[customerID] = recommended_products(customerID).index.tolist()
    return dictionary
[hello(customer) for customer in customerIDS]

dictionary

{83973: ['Baking Powder',
  'Guinea Fowl',
  'Cocoa Butter',
  'Thermometer Digital',
  'Muffin Chocolate Individual Wrap'],
 59399: ['Shrimp - Baby, Warm Water',
  'Knife Plastic - White',
  'Nantucket - Pomegranate Pear',
  'Snapple - Iced Tea Peach',
  'Wine - Fume Blanc Fetzer'],
 92168: ['Cod - Black Whole Fillet',
  'Cup - Translucent 7 Oz Clear',
  'Bacardi Breezer - Tropical',
  'Veal - Brisket, Provimi,bnls',
  'Snapple Lemon Tea'],
 49159: ['Cinnamon Buns Sticky',
  'Pasta - Penne, Rigate, Dry',
  'Cheese Cloth No 100',
  'Oil - Shortening,liqud, Fry',
  'Cookies - Assorted'],
 18441: ['Rabbit - Whole',
  'Juice - Apple Cider',
  'Anchovy Paste - 56 G Tube',
  'Sprouts - Baby Pea Tendrils',
  'Otomegusa Dashi Konbu'],
 22536: ['Kellogs All Bran Bars',
  'Appetizer - Sausage Rolls',
  'Bacardi Breezer - Tropical',
  'Cheese - Mozzarella',
  'Cheese Cloth No 100'],
 86028: ['Water - Mineral, Natural',
  'Salmon Steak - Cohoe 8 Oz',
  'Spinach - Baby',
  'Cheese - Brie, Triple C

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [16]:
recomendations_euclidean = pd.DataFrame(dictionary).T
recomendations_euclidean.head()

,0,1,2,3,4
83973,Baking Powder,Guinea Fowl,Cocoa Butter,Thermometer Digital,Muffin Chocolate Individual Wrap
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach,Wine - Fume Blanc Fetzer
92168,Cod - Black Whole Fillet,Cup - Translucent 7 Oz Clear,Bacardi Breezer - Tropical,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea
49159,Cinnamon Buns Sticky,"Pasta - Penne, Rigate, Dry",Cheese Cloth No 100,"Oil - Shortening,liqud, Fry",Cookies - Assorted
18441,Rabbit - Whole,Juice - Apple Cider,Anchovy Paste - 56 G Tube,Sprouts - Baby Pea Tendrils,Otomegusa Dashi Konbu


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [17]:
distances_normalised = pd.DataFrame(1/(1 + squareform(pdist(products.T, 'cosine'))), 
                        index=products.columns, columns=products.columns)

dictionary = dict()
[create_dictionary(customer) for customer in customerIDS]
recomendations_cosine = pd.DataFrame(dictionary).T
recomendations_cosine.head()
# Its different because the similar customers to our customers change. The angle is the same
# despite the amount of products each customer bought. With cosine if one customer buys
# 2 eggs and other customer buys 6 eggs they are more similar than with euclidean.

,0,1,2,3,4
83973,Wine - White Cab Sauv.on,Browning Caramel Glace,Tea - Herbal Sweet Dreams,Pastry - Choclate Baked,Rice - Jasmine Sented
59399,Ezy Change Mophandle,Anchovy Paste - 56 G Tube,Garlic - Peeled,Pastry - Choclate Baked,Broom - Corn
92168,Olive - Spread Tapenade,Pasta - Orecchiette,Wanton Wrap,Flavouring - Orange,Wiberg Super Cure
49159,Wine - White Cab Sauv.on,Wine - Chablis 2003 Champs,"Turnip - White, Organic",Guinea Fowl,Cinnamon Buns Sticky
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Halibut - Steaks,"Nut - Chestnuts, Whole",Wasabi Powder
